# Combine Fusion Program Outputs

## Set Input File Paths Below:
- Include a '/' at the end of the fusion caller and output file paths

In [ ]:
import pandas as pd
import os

In [ ]:
# 1. GFF3 Annotation File: 
gff3_input_annotation = '/path/to/gencode.v46.basic.annotation.gff3'


# 2. Set Input File Paths from LongGFs

# LongGF File Names for each parameter used
longgf_files = [
    'longgf.100-10-100.txt',
    'longgf.10-10-10.txt',
    'longgf.25-10-25.txt',
    'longgf.50-10-50.txt',
    'longgf.75-10-75.txt',
    'longgf.100-25-100.txt',
    'longgf.10-25-10.txt',
    'longgf.25-25-25.txt',
    'longgf.50-25-50.txt',
    'longgf.75-25-75.txt'
]

# Default output names remain the same for JAFFAL and FusionSeeker



# 3. Set Fusion Program Result Folder Paths:  
longgf_sample_path = '/path/to/longgf_outputs/'
jaffal_sample_path = '/path/to/jaffal_outputs/'
fusionseeker_sample_path = '/path/to/fusionseeker_outputs/'

# 4. Set Combined Summary Outputs Folder Path: 
output_sample_path = '/path/to/combined_fusion_calls_output_folder/' 
print('Output Directory: ', output_sample_path)

# Create the Output Folder Directory if it does not exist already: 
folder_name = f"{output_sample_path}"  
if not os.path.exists(folder_name):
    os.makedirs(folder_name)


### Process GFF3 Annotation File

In [ ]:
# 1. Set Reference GFF3 Annotation File 

# Process the GFF3 v46 annotation file for reference lookups, extracting key details.
# Load the GFF3 file into a DataFrame, skipping commented lines and using tab-delimited columns.
df = pd.read_csv(gff3_input_annotation, sep='\t', comment='#', header=None)

# Rename relevant columns for easier access to their contents.
df.rename(columns={0: 'chrom', 2: 'feature', 3: 'start', 4: 'end', 6: 'strand', 8: 'info'}, inplace=True)

# Extract 'gene_id' and 'gene_name' from the 'info' column using regex for mapping gene annotations
df['gene_id'] = df['info'].str.extract(r'gene_id=([^;]+)')
df['gene_name'] = df['info'].str.extract(r'gene_name=([^;]+)')

# Create a mapping of gene_name and chromosome to gene_id and strand for lookups of ref gene annotations
gene_name_map = {(x, y): (a, z) for x, y, z, a in zip(df.gene_name, df.chrom, df.gene_id, df.strand)}

# Define a function to retrieve gene_id and strand for a given gene_name and contig (chromosome).
# Returns None if the gene_name and contig combination is not found in the map.
def get_gene_name_map(gene_name, contig):
    try:
        return gene_name_map[(gene_name, contig)]
    except KeyError:
        return None, None

# Confirm that the GFF3 file has been processed and the DataFrame is ready for use.
print('GFF3 file is stored!')

# Display the DataFrame for verification.
df.head()

### LongGF Functions

In [ ]:
def calculate_breakpoint_distance(gene1_chrm, gene1_bkp, gene2_chrm, gene2_bkp):
    # Calculate the breakpoint distance
    if gene1_chrm == gene2_chrm:
        distance = abs(gene1_bkp - gene2_bkp)
    else:
        distance = float('inf')
    return distance


def process_longgf_output(longgf_output): #, gencode_data):
    fusions = []
    current_fusion = None
    read_id_process_lst = []
    gene1_strand_process_lst = []
    gene2_strand_process_lst = []
    gene1_position_process_lst = []
    gene2_position_process_lst = []
    process_lines = False

    with open(longgf_output, 'r') as file:
        for line in file:
            if line.startswith("Potential_fusion"):
                process_lines = True
                continue

            if process_lines:
                line_split = line.split()

                if line.startswith("GF"):
                    if current_fusion:
                        current_fusion.update({
                            'Read IDs': ",".join(read_id_process_lst),
                            'Gene 1 Strand': ",".join(gene1_strand_process_lst),
                            'Gene 2 Strand': ",".join(gene2_strand_process_lst),
                            'Gene 1 Input Position': ",".join(gene1_position_process_lst),
                            'Gene 2 Input Position': ",".join(gene2_position_process_lst),
                        })
                        fusions.append(current_fusion)

                    current_fusion = None
                    read_id_process_lst.clear()
                    gene1_strand_process_lst.clear()
                    gene2_strand_process_lst.clear()
                    gene1_position_process_lst.clear()
                    gene2_position_process_lst.clear()

                    support_reads = int(line_split[2])
                    fusion_name = line_split[1]
                    gene1_symbol, gene2_symbol = fusion_name.split(":")
                    current_fusion = {
                        'Fusion Name': fusion_name,
                        'Gene 1 Symbol': gene1_symbol,
                        'Gene 2 Symbol': gene2_symbol,
                        'Supporting Reads': support_reads,
                    }
                    
                    
                elif not line.startswith("SumGF"):
                    read_id_name = line.split('/')[1].split(':')[0]
                    read_id_process_lst.append(read_id_name)

                    gene1_strand = line.split("(")[1][0]
                    gene1_position = line.split(")")[0].split(":")[-1]
                    gene2_position = line.split(")")[1].split(":")[-1].split("/")[1]
                    gene2_strand = line.split("(")[2][0]

                    gene1_strand_process_lst.append(gene1_strand)
                    gene1_position_process_lst.append(gene1_position)
                    gene2_position_process_lst.append(gene2_position)
                    gene2_strand_process_lst.append(gene2_strand)

                elif line.startswith("SumGF"):
                    if current_fusion:
                        gene1_break = line_split[3]
                        gene2_break = line_split[4]

                        current_fusion['Gene 1 Breakpoint'] = gene1_break
                        current_fusion['Gene 2 Breakpoint'] = gene2_break

                        
                        
                        # Get strand information and gene ID using the updated helper function
                        gene1_strand, gene1_id = get_gene_name_map(current_fusion['Gene 1 Symbol'], gene1_break.split(':')[0])
                        current_fusion['Gene 1 Reference Strand'] = gene1_strand
                        current_fusion['Gene 1 ID'] = gene1_id

                        gene2_strand, gene2_id = get_gene_name_map(current_fusion['Gene 2 Symbol'], gene2_break.split(':')[0])
                        current_fusion['Gene 2 Reference Strand'] = gene2_strand
                        current_fusion['Gene 2 ID'] = gene2_id

                        # Calculate breakpoint distance
                        gene1_chrm = gene1_break.split(':')[0]
                        gene1_bkp = int(gene1_break.split(':')[1])
                        gene2_chrm = gene2_break.split(':')[0]
                        gene2_bkp = int(gene2_break.split(':')[1])

                        distance = calculate_breakpoint_distance(
                            gene1_chrm, gene1_bkp, gene2_chrm, gene2_bkp
                        )
                        current_fusion['Breakpoint Distance'] = distance
                        
                        
                        
    if current_fusion:
        current_fusion.update({
            'Read IDs': ",".join(read_id_process_lst),
            'Gene 1 Strand': ",".join(gene1_strand_process_lst),
            'Gene 2 Strand': ",".join(gene2_strand_process_lst),
            'Gene 1 Input Position': ",".join(gene1_position_process_lst),
            'Gene 2 Input Position': ",".join(gene2_position_process_lst),
        })
        fusions.append(current_fusion)

    longgf_df = pd.DataFrame(fusions)
    longgf_df['GF Program'] = 'LongGF'
    
    # Remove duplicate entries, keeping the one with the highest Supporting Reads
    longgf_df = longgf_df.sort_values('Supporting Reads', ascending=False)
    longgf_df = longgf_df.drop_duplicates(
        subset=['Fusion Name', 'Gene 1 ID', 'Gene 1 Breakpoint', 'Gene 2 ID', 'Gene 2 Breakpoint'],
        keep='first'
    )
    
    return longgf_df


# Check Strandness and Gene Positions, Flip if necessary

# Function definitions (previous code included)
def compare_strands(reference_strand, strands_list):
    strands_list = strands_list.split(',')
    comparison = [reference_strand == strand.strip() for strand in strands_list]
    return comparison

def compare_strand_lists(row):
    gene1_comparison = row['Gene 1 Strand Comparison Result']
    gene2_comparison = row['Gene 2 Strand Comparison Result']
    
    if gene1_comparison == gene2_comparison:
        return 'Consistent'
    else:
        return 'Inconsistent'

def compare_ranges(row):
    if row['Strand Comparison'] != 'Consistent':
        return 'Not Applicable'
    
    gene1_ranges = row['Gene 1 Input Position'].split(',')
    gene2_ranges = row['Gene 2 Input Position'].split(',')
    comparison_results = row['Gene 1 Strand Comparison Result']

    comparisons = []
    for i, result in enumerate(comparison_results):
        if i < len(gene1_ranges) and i < len(gene2_ranges):
            gene1_start = int(gene1_ranges[i].split('-')[0])
            gene2_start = int(gene2_ranges[i].split('-')[0])

            if result:
                lower_gene = 'Gene 1' if gene1_start < gene2_start else 'Gene 2'
                comparisons.append(lower_gene)
            else:
                higher_gene = 'Gene 1' if gene1_start > gene2_start else 'Gene 2'
                comparisons.append(higher_gene)

    result = 'Inconsistent' if 'Gene 1' in comparisons and 'Gene 2' in comparisons else ', '.join(set(comparisons))
    return result



# Orientation Check 
def flip_genes(row):
    if row['Range Comparisons'] == 'Gene 2':
        # Create a copy of the row
        flipped_row = row.copy()
        
        # Swap Gene 1 and Gene 2 related columns
        flipped_row['Gene 1 Symbol'] = row['Gene 2 Symbol']
        flipped_row['Gene 1 ID'] = row['Gene 2 ID']

        flipped_row['Gene 2 Symbol'] = row['Gene 1 Symbol']
        flipped_row['Gene 2 ID'] = row['Gene 1 ID']
        
        flipped_row['Gene 1 Breakpoint'] = row['Gene 2 Breakpoint']
        flipped_row['Gene 2 Breakpoint'] = row['Gene 1 Breakpoint']
        
        flipped_row['Gene 1 Reference Strand'] = row['Gene 2 Reference Strand']
        flipped_row['Gene 2 Reference Strand'] = row['Gene 1 Reference Strand']
        
        flipped_row['Gene 1 Strand'] = row['Gene 2 Strand']
        flipped_row['Gene 2 Strand'] = row['Gene 1 Strand']
        
        flipped_row['Gene 1 Input Position'] = row['Gene 2 Input Position']
        flipped_row['Gene 2 Input Position'] = row['Gene 1 Input Position']
        
        # Swap Fusion Name
        flipped_row['Fusion Name'] = f"{row['Gene 2 Symbol']}:{row['Gene 1 Symbol']}"
        
        return flipped_row
    else:
        return row
    
print("LongGF functions saved")

### JAFFAL Functions

In [ ]:
def compare_strand_lists(row):
    # Get strand and reference strand information
    gene1_strand = row['Gene 1 Strand']
    gene1_ref_strand = row['Gene 1 Reference Strand']
    gene2_strand = row['Gene 2 Strand']
    gene2_ref_strand = row['Gene 2 Reference Strand']
    
    # Determine if strands are consistent with reference strands
    gene1_match = (gene1_strand == gene1_ref_strand)
    gene2_match = (gene2_strand == gene2_ref_strand)
    
    # Determine if strands are opposite to their references
    gene1_opposite = (gene1_strand != gene1_ref_strand)
    gene2_opposite = (gene2_strand != gene2_ref_strand)
    
    if (gene1_match and gene2_match) or (gene1_opposite and gene2_opposite):
        return 'Consistent'
    else:
        return 'Inconsistent'
    
    
# Updated `process_new_file` function
def process_new_file(new_file):
    # Check if the file exists and is not empty
    if not os.path.exists(new_file) or os.stat(new_file).st_size == 0:
        print(f"Warning: Input file '{new_file}' is missing or empty.")
        
        # Define the column headers
        columns = [
            'Fusion Name', 'Gene 1 Symbol', 'Gene 1 ID', 'Gene 1 Breakpoint', 'Gene 1 Strand', 'Gene 1 Reference Strand',
            'Gene 2 Symbol', 'Gene 2 ID', 'Gene 2 Breakpoint', 'Gene 2 Strand', 'Gene 2 Reference Strand',
            'Breakpoint Distance', 'Supporting Reads', 'Read IDs', 'Strand Comparison', 'GF Program'
        ]
        
        # Create an empty DataFrame with just headers
        df = pd.DataFrame(columns=columns)
        return df
    
    # Read the new file into a DataFrame
    df = pd.read_csv(new_file, sep=',')  # Adjust sep based on the file's delimiter

    # Drop unnecessary columns
    df = df.drop(columns=['sample', 'known', 'spanning pairs', 'rearrangement', 'contig break', 'inframe', 'aligns', 'classification'])

    # Rename columns
    df = df.rename(columns={'fusion genes': 'Fusion Name', 'contig': 'Read IDs', 'spanning reads': 'Supporting Reads', 'gap (kb)': 'Breakpoint Distance'})

    # Extract 'Gene 1 Symbol' and 'Gene 2 Symbol' from 'Fusion Name'
    df[['Gene 1 Symbol', 'Gene 2 Symbol']] = df['Fusion Name'].str.split(':', expand=True)

    # Get reference strand and gene ID for both genes
    gene1_strand_lst, gene1_id_lst, gene2_strand_lst, gene2_id_lst = [], [], [], []
    
    for _, row in df.iterrows():
        gene1_symbol, contig1 = row['Gene 1 Symbol'], row['chrom1']
        gene1_strand, gene1_id = get_gene_name_map(gene1_symbol, contig1)
        gene1_strand_lst.append(gene1_strand)
        gene1_id_lst.append(gene1_id)

        gene2_symbol, contig2 = row['Gene 2 Symbol'], row['chrom2']
        gene2_strand, gene2_id = get_gene_name_map(gene2_symbol, contig2)
        gene2_strand_lst.append(gene2_strand)
        gene2_id_lst.append(gene2_id)

    # Add new columns to the DataFrame
    df['Gene 1 ID'] = gene1_id_lst
    df['Gene 1 Breakpoint'] = df['chrom1'] + ':' + df['base1'].astype(str)
    df['Gene 1 Strand'] = df['strand1']
    df['Gene 1 Reference Strand'] = gene1_strand_lst

    df['Gene 2 ID'] = gene2_id_lst
    df['Gene 2 Breakpoint'] = df['chrom2'] + ':' + df['base2'].astype(str)
    df['Gene 2 Strand'] = df['strand2']
    df['Gene 2 Reference Strand'] = gene2_strand_lst

    # Add strand comparison columns
    df['Strand Comparison'] = df.apply(compare_strand_lists, axis=1)

    df['GF Program'] = 'JAFFAL'

    # Reorder columns to match the required format
    df = df[[
        'Fusion Name', 'Gene 1 Symbol', 'Gene 1 ID', 'Gene 1 Breakpoint', 'Gene 1 Strand', 'Gene 1 Reference Strand',
        'Gene 2 Symbol', 'Gene 2 ID', 'Gene 2 Breakpoint', 'Gene 2 Strand', 'Gene 2 Reference Strand',
        'Breakpoint Distance', 'Supporting Reads', 'Read IDs', 'Strand Comparison', 'GF Program'
    ]]

    return df  

print("JAFFAL functions saved")

### FusionSeeker

In [ ]:
def process_fusionseeker_summary(input_fusionseeker_file): 
    
    # Step 1: Read the file with a header
    summary_fusionseeker_df = pd.read_csv(input_fusionseeker_file, sep="\t")

    # Check if the file only contains the header and no data rows
    if summary_fusionseeker_df.shape[0] == 0:  # If there are no rows (only the header)
        print("\nFusionSeeker summary file has no data. Creating an empty summary file with headers only.")
        empty_columns = [
            'Fusion Name', 'Gene 1 Symbol', 'Gene 1 ID', 'Gene 1 Breakpoint', 'Gene 1 Strand', 'Gene 1 Reference Strand',
            'Gene 2 Symbol', 'Gene 2 ID', 'Gene 2 Breakpoint', 'Gene 2 Strand', 'Gene 2 Reference Strand',
            'Breakpoint Distance', 'Supporting Reads', 'Read IDs', 'Strand Comparison', 'GF Program'
        ]
        summary_fusionseeker_df = pd.DataFrame(columns=empty_columns)
        
        
        
    else:
        # Step 2: Rename columns to match your desired naming convention
        summary_fusionseeker_df.rename(columns={
            'Gene1': 'Gene 1 Symbol',
            'Gene2': 'Gene 2 Symbol',
            'Chrom1': 'Gene1 Chromosome',
            'Breakpoint1': 'Gene1 Breakpoint',
            'Chrom2': 'Gene2 Chromosome',
            'Breakpoint2': 'Gene2 Breakpoint',
            'NumSupp': 'Supporting Reads',
            'SupportingReads': 'Read IDs'
        }, inplace=True)
        
        
        # Step 3: Initialize lists for gene IDs and strand information
        gene1_strand_lst, gene1_id_lst = [], []
        gene2_strand_lst, gene2_id_lst = [], []

        # Populate the lists with data from get_gene_name_map
        for _, row in summary_fusionseeker_df.iterrows():
            gene1_strand, gene1_id = get_gene_name_map(row['Gene 1 Symbol'], row['Gene1 Chromosome'])
            gene1_strand_lst.append(gene1_strand)
            gene1_id_lst.append(gene1_id)

            gene2_strand, gene2_id = get_gene_name_map(row['Gene 2 Symbol'], row['Gene2 Chromosome'])
            gene2_strand_lst.append(gene2_strand)
            gene2_id_lst.append(gene2_id)

        # Step 4: Add and rename columns based on the specified format
        summary_fusionseeker_df['Gene 1 ID'] = gene1_id_lst
        summary_fusionseeker_df['Gene 1 Strand'] = gene1_strand_lst
        summary_fusionseeker_df['Gene 1 Reference Strand'] = gene1_strand_lst
        summary_fusionseeker_df['Gene 2 ID'] = gene2_id_lst
        summary_fusionseeker_df['Gene 2 Strand'] = gene2_strand_lst
        summary_fusionseeker_df['Gene 2 Reference Strand'] = gene2_strand_lst

        # Combine chromosome and breakpoint columns for breakpoints
        summary_fusionseeker_df['Gene 1 Breakpoint'] = (
            summary_fusionseeker_df['Gene1 Chromosome'] + ':' + summary_fusionseeker_df['Gene1 Breakpoint'].astype(str)
        )
        summary_fusionseeker_df['Gene 2 Breakpoint'] = (
            summary_fusionseeker_df['Gene2 Chromosome'] + ':' + summary_fusionseeker_df['Gene2 Breakpoint'].astype(str)
        )

        # Strand Comparison
        summary_fusionseeker_df['Strand Comparison'] = summary_fusionseeker_df.apply(compare_strand_lists, axis=1)
        
        # Breakpoint Distance
        summary_fusionseeker_df['Breakpoint Distance'] = summary_fusionseeker_df.apply(
            lambda row: calculate_breakpoint_distance(
                row['Gene1 Chromosome'], int(row['Gene1 Breakpoint']), 
                row['Gene2 Chromosome'], int(row['Gene2 Breakpoint'])
            ), axis=1
        )

        # Add GF Program and Fusion Name columns
        summary_fusionseeker_df['GF Program'] = 'FusionSeeker'
        summary_fusionseeker_df['Fusion Name'] = summary_fusionseeker_df['Gene 1 Symbol'] + ':' + summary_fusionseeker_df['Gene 2 Symbol']

        # Step 5: Reorder columns
        summary_fusionseeker_df = summary_fusionseeker_df[[
            'Fusion Name', 'Gene 1 Symbol', 'Gene 1 ID', 'Gene 1 Breakpoint', 'Gene 1 Strand', 'Gene 1 Reference Strand',
            'Gene 2 Symbol', 'Gene 2 ID', 'Gene 2 Breakpoint', 'Gene 2 Strand', 'Gene 2 Reference Strand',
            'Breakpoint Distance', 'Supporting Reads', 'Read IDs', 'Strand Comparison', 'GF Program'
        ]]

    # Return the final DataFrame for any further processing or display
    return summary_fusionseeker_df

print("FusionSeeker functions saved")

## Combine Fusion Calls from LongGF, JAFFAL, and FusionSeeker

In [ ]:
# Combine Output Fusion Files from Each Program

######### LONGGF #########
print("Processing LongGF Output: ")
all_dfs = []
for longgf_input in longgf_files:
    input_file_name = longgf_sample_path + longgf_input

    # Process the file
    longgf_df = process_longgf_output(input_file_name) 

    longgf_df['Gene 1 Strand Comparison Result'] = longgf_df.apply(
        lambda row: compare_strands(row['Gene 1 Reference Strand'], row['Gene 1 Strand']), axis=1)

    longgf_df['Gene 2 Strand Comparison Result'] = longgf_df.apply(
        lambda row: compare_strands(row['Gene 2 Reference Strand'], row['Gene 2 Strand']), axis=1)

    longgf_df['Strand Comparison'] = longgf_df.apply(compare_strand_lists, axis=1)
    longgf_df['Range Comparisons'] = longgf_df.apply(compare_ranges, axis=1)

    # Orientation Check Function
    flipped_df = longgf_df.apply(flip_genes, axis=1)


    # Output Summary of individual longgf file with proper orientation and the desired column order
    desired_order = [
        'Fusion Name','Gene 1 Symbol', 'Gene 1 ID', 'Gene 1 Strand', 
        'Gene 1 Reference Strand', 'Gene 1 Breakpoint', 'Gene 1 Input Position', 
        'Gene 2 Symbol', 'Gene 2 ID', 'Gene 2 Strand',
        'Gene 2 Reference Strand', 'Gene 2 Breakpoint', 'Gene 2 Input Position',
        'Breakpoint Distance', 'Supporting Reads', 'Read IDs', 'GF Program', 'Strand Comparison']

    # Reorder columns by direct indexing and append to all_dfs list
    longgf_summary_output = flipped_df[desired_order]
    all_dfs.append(longgf_summary_output)

# Concatenate all DataFrames together
concatenated_longgf_df = pd.concat(all_dfs, ignore_index=True)
print('Initial Number of LongGF fusions detected across all parameters: ', concatenated_longgf_df.shape[0])

# Drop duplicates and Retain Entry with the Highest Supporting Reads
# Drop duplicates based on: gene name, id, breakpoint for both genes
deduplicated_longgf_df = concatenated_longgf_df.loc[
    concatenated_longgf_df.groupby(
        ['Gene 1 Symbol', 'Gene 1 Breakpoint', 'Gene 1 ID', 
         'Gene 2 Symbol', 'Gene 2 Breakpoint', 'Gene 2 ID'])['Supporting Reads'].idxmax()]

# Reset index and report how many remaining 
deduplicated_longgf_df = deduplicated_longgf_df.reset_index(drop=True)
print('Number of LongGF fusions detected across all parameters after dropping duplicates: ', deduplicated_longgf_df.shape[0])

# Output the final concatenated DataFrame
output_file_lgf = f"{output_sample_path}summary_longgf.tsv"
deduplicated_longgf_df.to_csv(output_file_lgf, sep='\t', index=False)
print(f"\nSummary file saved: {output_file_lgf}")


In [ ]:
######### JAFFAL #########
print("Processing JAFFAL Output: ")

input_jaffal_file = jaffal_sample_path + 'jaffa_results.csv'    
summary_JAFFAL_output = process_new_file(input_jaffal_file)
print('Number of JAFFAL fusions detected: ', summary_JAFFAL_output.shape[0], '\n')

output_file_jaffal = f"{output_sample_path}summary_jaffal.tsv"
summary_JAFFAL_output.to_csv(output_file_jaffal, sep='\t', index=False)
print(f"Summary file saved: {output_file_jaffal}")


In [ ]:
######### FusionSeeker #########
print("Processing FusionSeeker Output: ")

input_fusionseeker_file = fusionseeker_sample_path + 'confident_genefusion.txt'
summary_FS_output = process_fusionseeker_summary(input_fusionseeker_file)
print('Number of FusionSeeker fusions detected: ', summary_FS_output.shape[0])

output_file_fusionseeker = f"{output_sample_path}summary_fusionseeker.tsv"
summary_FS_output.to_csv(output_file_fusionseeker, sep='\t', index=False)
print(f"\nSummary file saved: {output_file_fusionseeker}")


In [ ]:
######### Combine All Program Outputs Together #########

summary_df_to_check = pd.concat([concatenated_longgf_df, summary_JAFFAL_output, summary_FS_output], axis=0)
print('Total Number of Fusions Detected Across all 3 Programs: ', summary_df_to_check.shape[0])

output_file_fusionseeker = f"{output_sample_path}summary_all_fusions.tsv"
summary_df_to_check.to_csv(output_file_fusionseeker, sep='\t', index=False)
print(f"\nSummary file saved: {output_file_fusionseeker}")